In [1]:
import pandas as pd

df = pd.DataFrame([{'Name': 'Chris', 'Item Purchased': 'Sponge', 'Cost': 22.50},
                   {'Name': 'Kevyn', 'Item Purchased': 'Kitty Litter', 'Cost': 2.50},
                   {'Name': 'Filip', 'Item Purchased': 'Spoon', 'Cost': 5.00}],
                  index = ['Store 1', 'Store 1', 'Store 2'])

df

,Cost,Item Purchased,Name
Store 1,22.5,Sponge,Chris
Store 1,2.5,Kitty Litter,Kevyn
Store 2,5.0,Spoon,Filip


Pass in a scalar value to add a new column with default value:

In [2]:
df['Delivered'] = True
df

,Cost,Item Purchased,Name,Delivered
Store 1,22.5,Sponge,Chris,True
Store 1,2.5,Kitty Litter,Kevyn,True
Store 2,5.0,Spoon,Filip,True


To assign a different value for every row we could pass in list of values, **which is long enough**, then Pandas will unpack them and assign them to the rows:

In [3]:
df['Feedback'] = ['Positive', None, 'Negative']
df

,Cost,Item Purchased,Name,Delivered,Feedback
Store 1,22.5,Sponge,Chris,True,Positive
Store 1,2.5,Kitty Litter,Kevyn,True,None
Store 2,5.0,Spoon,Filip,True,Negative


If each rows has a unique index, then we could assign the new column identifier to the series. In this approach we can ignore some items:

In [4]:
df.reset_index(inplace=True)
df['Date'] = pd.Series( {0: 'Dec 1', 2: 'mid-May'} )
df

,index,Cost,Item Purchased,Name,Delivered,Feedback,Date
0,Store 1,22.5,Sponge,Chris,True,Positive,Dec 1
1,Store 1,2.5,Kitty Litter,Kevyn,True,None,NaN
2,Store 2,5.0,Spoon,Filip,True,Negative,mid-May


## Merging DataFrames

![Operations on Sets](_files/Operations_on_Sets.png)

Create two DataFrames with some overlap. Both are indexed along the value we want to merge them on - "Name": 

In [5]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df.set_index('Name', inplace=True)
staff_df

,Role
Name,
Kelly,Director of HR
Sally,Course liasion
James,Grader


In [6]:
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])
student_df.set_index('Name', inplace=True)
student_df

,School
Name,
James,Business
Mike,Law
Sally,Engineering


`outer`: Use **Union** of keys from both frames, similar to a SQL **Full Outer Join**, sort keys lexicographically:

In [7]:
pd.merge(staff_df, student_df,
         how='outer', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


`inner`: Use **Intersection** of keys from both frames, similar to a SQL **Inner Join**, preserve the order of the left keys:

In [8]:
pd.merge(staff_df, student_df,
         how='inner', left_index=True, right_index=True)

,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


The other common use cases when merging DataFrames - **set Addition** (Left or Right Join).

`left`: Use only keys from left frame, similar to a SQL **Left Join**, preserve key order:

In [9]:
pd.merge(staff_df, student_df,
         how='left', left_index=True, right_index=True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


`right`: Use only keys from right frame, similar to a SQL **Right Join**, preserve key order:

In [10]:
pd.merge(staff_df, student_df,
         how='right', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


We can use columns instead of indices to join on:

In [11]:
staff_df.reset_index(inplace=True)
student_df.reset_index(inplace=True)

pd.merge(staff_df, student_df,
         how='left', left_on='Name', right_on='Name')

,Name,Role,School
0,Kelly,Director of HR,NaN
1,Sally,Course liasion,Engineering
2,James,Grader,Business


Also we can use index and column together:

In [12]:
products_df = pd.DataFrame([{'Product ID': '111', 'Price': '5'},
                            {'Product ID': '112', 'Price': '1'},
                            {'Product ID': '113', 'Price': '7'}])
products_df.set_index('Product ID', inplace=True)

invoices_df = pd.DataFrame([{'Product ID': '111', 'Customer': 'Bob'},
                            {'Product ID': '112', 'Customer': 'Kate'},
                            {'Product ID': '113', 'Customer': 'James'}])

pd.merge(products_df, invoices_df, how='outer', left_index=True, right_on='Product ID')

,Price,Customer,Product ID
0,5,Bob,111
1,1,Kate,112
2,7,James,113


**Conflicts between the DataFrames**

The "Director of HR" is on "State Street", while the two students are on "Washington Avenue". But for the student DataFrame, the location information is actually their home address.

The `merge` function preserves this information, but appends an "\_x" or "\_y" to help differentiate between which index went with which column of data. The "\_x" is always the left DataFrame information, and the "\_y" is always the right DataFrame information. And you could control the names of "\_x" and "\_y" with additional parameters if you want to:

In [13]:
staff_df = pd.DataFrame([
    {'Name': 'Kelly', 'Role': 'Director of HR', 'Location': 'State Street'},
    {'Name': 'Sally', 'Role': 'Course liasion', 'Location': 'Washington Avenue'},
    {'Name': 'James', 'Role': 'Grader', 'Location': 'Washington Avenue'}
])

student_df = pd.DataFrame([
    {'Name': 'James', 'School': 'Business', 'Location': '1024 Billiard Avenue'},
    {'Name': 'Mike', 'School': 'Law', 'Location': 'Fraternity House #22'},
    {'Name': 'Sally', 'School': 'Engineering', 'Location': '512 Wilson Crescent'}
])

pd.merge(staff_df, student_df, how='left', left_on='Name', right_on='Name')

,Location_x,Name,Role,Location_y,School
0,State Street,Kelly,Director of HR,NaN,NaN
1,Washington Avenue,Sally,Course liasion,512 Wilson Crescent,Engineering
2,Washington Avenue,James,Grader,1024 Billiard Avenue,Business


**Multi-indexing and multiple columns**

For example the first name for students and staff might overlap, but the last name might not. In this case, we use a list of the multiple columns that should be used to join keys on the `left_on` and `right_on` parameters:

In [14]:
staff_df = pd.DataFrame([
    {'First Name': 'Kelly', 'Last Name': 'Desjardins', 'Role': 'Director of HR'},
    {'First Name': 'Sally', 'Last Name': 'Brooks', 'Role': 'Course liasion'},
    {'First Name': 'James', 'Last Name': 'Wilde', 'Role': 'Grader'}
])

student_df = pd.DataFrame([
    {'First Name': 'James', 'Last Name': 'Hammond', 'School': 'Business'},
    {'First Name': 'Mike', 'Last Name': 'Smith', 'School': 'Law'},
    {'First Name': 'Sally', 'Last Name': 'Brooks', 'School': 'Engineering'}
])

In [15]:
staff_df

,First Name,Last Name,Role
0,Kelly,Desjardins,Director of HR
1,Sally,Brooks,Course liasion
2,James,Wilde,Grader


In [16]:
student_df

,First Name,Last Name,School
0,James,Hammond,Business
1,Mike,Smith,Law
2,Sally,Brooks,Engineering


In [17]:
pd.merge(staff_df, student_df, how='inner',
         left_on=['First Name','Last Name'],
         right_on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering
